DEFINITION LAMBDA, w, P, pi

In [ ]:

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

Lambda = np.array([
    [0, 2/5, 1/5, 0, 0],
    [0, 0, 3/4, 1/4, 0],
    [1/2, 0, 0, 1/3, 0],
    [0, 0, 1/3, 0, 2/3],
    [0, 1/3, 0, 1/3, 0]
])

# clock rates w
w = np.sum(Lambda, axis=1)

# contruct the P matrix (instead of P_bar) P = D^-1 * Lambda
D = np.diag(w)
P = np.linalg.inv(D) @ Lambda

# compute dominant eigenvector
eigenvalues, eigenvectors = np.linalg.eig(P.T)
# find index of dominant eigenvalue
index_of_unity_eigenvalue = np.where(np.isclose(eigenvalues, 1))[0][0]
# analitically the probability distribution = eigenvector associated
# to the dominant eigenvalue normalized
dominant_eigenvector = eigenvectors[:, index_of_unity_eigenvalue].real
pi_bar = dominant_eigenvector / np.sum(dominant_eigenvector)
print("pi_bar = ", pi_bar)

# number of states, usefull later
n_states = len(pi_bar)

# print("\nw = ", w)
# print("\nD = ", D)
# print("\nP = ", P)
# print("\nP_bar = ", P_bar)

HOMEWORK 2 ES 1.A

In [ ]:
import numpy as np

# define return node, node b = 2
return_node = 2

# number of simulations and number of iteration in each of theme
n_simulations = 1000
n_steps = 10000

# save return time of each simulation
return_times_list = []

for _ in range(n_simulations):

    # pos will keep trace of the visited states
    pos = np.zeros(n_steps, dtype=int)-1

    # start from state b
    pos[0] = return_node

    # transition_times will store the time instants at which jumps/transitions happen
    transition_times = np.zeros(n_steps)

    for t in range(1, n_steps):
        # the next state to visit will be extracted according to the probabilities
        # stored in the row of P corresponding to the current state.
        pos[t] = np.random.choice(n_states, p=P[pos[t-1],:])

        # compute the waiting time to the next transition
        # t_next = -log(u)/r
        # NOTE: we use the rate w[pos[t]] of the clock of the current position.
        # So it's a Local Clock
        t_next = -np.log(np.random.rand())/w[pos[t-1]]
        # store the time instant of the current transition
        transition_times[t] = transition_times[t-1] + t_next

        # If the particle returns to b, stop the simulation
        if pos[t] == return_node:
            break

    # find index of pos where particle returned to b
    index_return_node = np.where(pos == return_node)[0]

    # if particle never returned skip this result
    if len(index_return_node) == 2:
        # add transitoin time to return time list
        return_times_list.append(transition_times[index_return_node[1]])

# compute mean
mean_return_time_overall = np.mean(return_times_list)

print(f"Average time on {n_simulations} simulations: {mean_return_time_overall}")


HOMEWORK 2
ES 1.B

In [ ]:
# compute analitically the expected retrun time
E = 1/(w*pi_bar)
print("Expected return time on b is ", E[return_node])

HOMEWORK 2
ES 1
PUNTO C

In [ ]:
import numpy as np

# Number of simulations
n_simulations = 1000
# Set the number of steps in the simulation
n_steps = 10000
# Start and end nodes
start_node = 0
end_node = 4

# List to store return times for each simulation
return_times_list = []

for _ in range(n_simulations):

    # pos will keep trace of the visited states
    pos = np.zeros(n_steps, dtype=int)-1
    # Start from the initial node
    pos[0] = start_node
    # transition_times will store the time instants at which jumps/transitions happen
    transition_times = np.zeros(n_steps)
    # Random time to wait for the next transition
    # Drawn according to its distribution, discussed in Remark 2
    # NOTE: In the formula for t_next, we use the rate of the clock of the current state, in this case w[start_node].
    t_next = -np.log(np.random.rand())/w[start_node]

    for t in range(1, n_steps):
        # The next state to visit will be extracted according to the probabilities
        # stored in the row of P corresponding to the current state.
        pos[t] = np.random.choice(n_states, p=P[pos[t-1],:])
        # Store the time instant of the current transition
        t_next = -np.log(np.random.rand())/w[pos[t-1]]
        transition_times[t] = transition_times[t-1] + t_next
        # If the particle reaches the destination node, stop the simulation
        if pos[t] == end_node:
            break


    # Find the indices of the times when the particle leaves the start node
    index_end_node = np.where(pos == end_node)[0]

    # If the particle reach the end node
    if len(index_end_node) != 0:
        # Add the return time to the list
        return_times_list.append(transition_times[index_end_node])

# Calculate the average return time over all simulations
mean_return_time_overall = np.mean(return_times_list)

# Print the average return time
print(f"Average time to move from node {start_node} to node {end_node} over {n_simulations} simulations: {mean_return_time_overall}")


HOMEWORK 2
ES 1
PUNTO D

In [ ]:
import sympy as sp
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

# Define symbolic variables
tau_i=[]
for node in range(n_states):
    tau_i.append(sp.symbols(f'tau_{node}_{end_node}'))


# Define the system of equations
equation=[ sp.Eq(tau_i[end_node] , 0) ]

for node in range(n_states):
    if node != end_node:
        equation.append(sp.Eq(tau_i[node] , 1/w[node] + sum([P[node][j] * tau_i[j] for j in range(n_states)]) ))

# Solve the system of equations
solution = sp.solve(equation, tau_i)

# Print the solution
print(f"Average time to move from node {start_node} to node {end_node} is : {solution[tau_i[start_node]]}")

HOMEWORK 2 ES 1.E

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import sympy as sp


W = Lambda
#degree vector and diagonal degree matrix
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
# contruct of P = D^-1 * W
P = np.linalg.inv(D) @ W

print("P =\n",P)


# G = nx.DiGraph()
# G.add_edges_from([(0,1),(0,2),(1,2),(1,3),(2,0),(2,3),(3,4),(3,2),(4,1),(4,3)])
# labels of nodes are couples: (column,row)
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True)
# CG = nx.algorithms.components.condensation(G)
# # nx.draw(CG)


In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import sympy as sp

# compute dominant eigenvector
eigenvalues, eigenvectors = np.linalg.eig(P.T)
# find index of dominant eigenvalue
index_of_unity_eigenvalue = np.where(np.isclose(eigenvalues, 1))[0][0]
# analitically the probability distribution = eigenvector associated
# to the dominant eigenvalue normalized
dominant_eigenvector = eigenvectors[:, index_of_unity_eigenvalue].real
pi = dominant_eigenvector / np.sum(dominant_eigenvector)

print("Stationary vector pi = ", pi)

# initial condition
x0 = [1,0,0.2,0.5,1]
print("\nx0 = ",x0)

# simulation, x(t+1) = P * x(t)
x = x0
for n in range(99):
    x = P @ x
print("x(100) = ", x)

consensus = pi @ x0
print("consensus = ", consensus)

# different initial condition
x0 = [2,3,5.2,0.4,1.1]
print("\nx0 = ",x0)

# simulation, x(t+1) = P * x(t)
x = x0
for n in range(99):
    x = P @ x
print("x(100) = ", x)

consensus = pi @ x0
print("consensus = ", consensus)

HOMEWORK 2
ES 1.F

In [ ]:
# compute dominant eigenvector
eigenvalues, eigenvectors = np.linalg.eig(P.T)
# find index of dominant eigenvalue
index_of_unity_eigenvalue = np.where(np.isclose(eigenvalues, 1))[0][0]
# analitically the probability distribution = eigenvector associated
# to the dominant eigenvalue normalized
dominant_eigenvector = eigenvectors[:, index_of_unity_eigenvalue].real
pi = dominant_eigenvector / np.sum(dominant_eigenvector)
print("Stationary vector pi = ", pi)


#definition of the initial condition vector
x_real = [0,0,0,0,0]

xn = [0,0,0,0,0]
#definition of the variance vector
variance = [2,1,1,1,2]
#definition of the standard deviation 
dev = np.sqrt(variance)
#compute the noisy vector
for i in range(len(xn)):
    xn[i] = x_real[i] + np.random.normal(0, dev[i])

print("x=",x_real)
print("x_noise=",xn)


pi_square = np.square(pi)
#compute the variance of the consensus
var_consensus = pi_square @ variance
print("Consensus variance:",var_consensus)

# compute the noisy consensus
consensus_noise = pi @ xn
print("\nConsensus with noise:",consensus_noise)
x = xn
for n in range(99):
    x = P @ x
print("simulation result = ", x)

# compute the real consensus
consensus_real = pi @ x_real
pi_square = np.square(pi)
print("Consensus without noise:",consensus_real)

x = x_real
for n in range(99):
    x = P @ x
print("simulation result = ", x)

HOMEWORK 2
ES 1.G

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
G = nx.DiGraph()
G.add_edges_from([(0,1),(0,2),(1,2),(1,3),(2,0),(2,3),(3,4),(3,2)])

# labels of nodes are couples: (column,row)
pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True)
CG = nx.algorithms.components.condensation(G)
# nx.draw(CG)

In [ ]:
W = Lambda.copy()
# definition of the new matrix W
W[4,1] = 0
W[4,3] = 0
W[4,4] = 1 #self loop
print('w : \n', W)

# compute of the matrix W, D, P
w = np.sum(W, axis=1)
D = np.diag(w)
P = np.linalg.inv(D) @ W

# compute the matrix W and E of the average dynamics with input
Q = P[:4 , :4]
E = P[:4 , 4]

#inverse of the matrix (I-Q)
I_Q_inv = np.linalg.inv(np.eye(Q.shape[0]) - Q)

dev = 1
# definition of the initial condition and the input
x0 = [np.random.normal(0, dev) for _ in range(n_states)]
u = x0[-1]

x = np.zeros((5,))
# compute the average dyamics with input
x[:4] =  I_Q_inv @ E * u
x[4] = u
print(f'Consensus : {x}\nReached with initial conditions : {x0},\nu : {x0[-1]:.4f} ')


x = x0
for n in range(99):
    x = P @ x
print("simulation result = ", x)

# degrees = np.sum(W,axis=1)
# D = np.diag(degrees)
# L = D - W
# L @ x

HOMEWORK 2
ES 1.H

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

G = nx.DiGraph()
G.add_edges_from([(0,1),(0,2),(1,2),(1,3),(2,0),(2,3),(3,4),(4,3)])

# labels of nodes are couples: (column,row)
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True)

In [ ]:
CG = nx.algorithms.components.condensation(G)
# nx.draw(CG)

In [ ]:
W = np.array([
    [0, 2/5, 1/5, 0, 0],
    [0, 0, 3/4, 1/4, 0],
    [1/2, 0, 0, 1/3, 0],
    [0, 0, 0, 0, 2/3],
    [0, 0, 0, 1/3, 0]
])
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W
print("P=",P)
x0 = [1,0,0.2,0.5,1]
x = x0
# compute of the x after 100 iterations
for n in range(100):
    x = P @ x
print("x(100):", x, "\n")